In [1]:
import json 
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display

# Playing around with some directory code

current_dir = os.getcwd()
rel_path = os.path.join("data", "daily_production")
json_path = os.path.join(rel_path, "BRU")
contents = os.listdir(rel_path)
basename = os.path.basename(rel_path)
dirname = os.path.dirname(rel_path)

# Playing around with some JSON code
def read_json_files(folder_path):
    # Create an empty DataFrame with specific columns
    df_dict = {
    'DoW': [],
    'hour': [],
    'minute': [],
    'date': [],
    'maintenance': [],
    'prod_loss': [],
    'prod_loss_perc': [],
    'production': []
    }
    df = pd.DataFrame(df_dict)

    # Iterate over each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)
            
            # Read the JSON file
            with open(file_path, 'r') as file:
                data = json.load(file)

                # Convert the dictionary to a list of dictionaries
                data_list = [data]
                
                # Convert the list of dictionaries to a dataframe
                temp_df = pd.DataFrame(data_list)
                
                # Append the temp_df to the DataFrame
                df = pd.concat([df, temp_df], ignore_index=True)
    
    return df

# Define the relative path to the folder
rel_path = os.path.join("data", "daily_production", "BRU")

# Read JSON files and populate the DataFrame
data_frame = read_json_files(rel_path)
display(data_frame)

# Printing out results from directory code

print(contents)
print(basename)
print(dirname)
print(os.listdir(json_path))

,DoW,hour,minute,date,maintenance,prod_loss,prod_loss_perc,production
0,Tuesday,1.0,0.0,09-22-2020 00:00:00.0000,No,225.0,18.0,1019.0
1,Monday,1.0,0.0,01-22-2018 00:00:00.0000,No,255.0,20.0,989.0
2,Wednesday,1.0,0.0,08-21-2019 00:00:00.0000,No,138.0,11.0,1106.0
3,Tuesday,1.0,0.0,08-21-2018 00:00:00.0000,No,302.0,24.0,942.0
4,Wednesday,1.0,0.0,09-22-2021 00:00:00.0000,No,211.0,17.0,1033.0
...,...,...,...,...,...,...,...,...
1825,Friday,1.0,0.0,07-29-2022 00:00:00.0000,No,157,13,1087
1826,Monday,1.0,0.0,07-20-2020 00:00:00.0000,No,289,23,955
1827,Sunday,1.0,0.0,06-23-2019 00:00:00.0000,No,140,11,1104
1828,Saturday,1.0,0.0,06-23-2018 00:00:00.0000,No,204,16,1040


['BRU', 'STO']
daily_production
data
['20200922.json', '20180122.json', '20190821.json', '20180821.json', '20210922.json', '20190122.json', '20181226.json', '20191226.json', '20220301.json', '20210308.json', '20220614.json', '20200308.json', '20221005.json', '20190525.json', '20200426.json', '20210426.json', '20180525.json', '20200830.json', '20210830.json', '20220706.json', '20180319.json', '20221117.json', '20190319.json', '20220213.json', '20201221.json', '20210430.json', '20200430.json', '20211221.json', '20221228.json', '20210826.json', '20180925.json', '20200125.json', '20190925.json', '20210125.json', '20200826.json', '20221101.json', '20220710.json', '20220205.json', '20201108.json', '20210719.json', '20200719.json', '20211108.json', '20180421.json', '20191230.json', '20210522.json', '20181230.json', '20200522.json', '20190421.json', '20180708.json', '20191119.json', '20220317.json', '20181119.json', '20190708.json', '20221013.json', '20220602.json', '20210109.json', '20190909.